<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/TrajectorOfSphereWithDragCrisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

In [245]:
def Cd(Re):
    a=Re/5
    b=Re/2.63e5
    c=Re/1e6
    return 24/Re + 2.6*a/(1+a**1.52) +0.411*b**(-7.94)/(1+b**(-8.)) + 0.25*c/(1+c)

In [246]:
x = np.logspace(2,7,100)
Cd_vals = Cd(x)

In [247]:
fig = make_subplots()
fig.add_scatter(x=x,y=Cd_vals, mode='lines')
fig.update_xaxes(type='log')
fig.update_layout(width=800,height=600, template='plotly_dark')

In [250]:
rho = 1.29 #kg/m3
mu = 1.8e-5
d = 0.22
A = np.pi*(d**2)/4
g = 9.81
m= 0.42

def rhs(t, xv):
    v=xv[2:]
    v_mag = np.linalg.norm(v)
    Re = rho*v_mag*d/mu
    dxdt = v
    dvdt = np.array([0, -g]) - rho*Cd(Re)*A*v_mag/2*v/m
    return np.concatenate([dxdt,dvdt])

def event(t, xv):
    return xv[1]

event.terminal=True
event.direction=-1

In [251]:
v_mag_mph = 80
angle = 45
v_mag = v_mag_mph * 0.44704
theta = angle*np.pi/180
v = v_mag*np.array([np.cos(theta), np.sin(theta)])

In [255]:
tend=10
res = solve_ivp(rhs, (0,tend), np.concatenate([[0,0],v]), method='Radau', dense_output=True, events=event)

In [257]:
t_plot = np.linspace(0,res.t_events[0][0],50)
x,y=res.sol(t_plot)[:2]
fig2=make_subplots()
fig2.add_scatter(x=x,y=y,mode='markers')
fig2.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig2.update_layout(width=800,height=600, template='plotly_dark')